In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv('zomato_bglr.csv')

In [3]:
data.drop(['url','dish_liked','phone'],axis=1,inplace=True)

In [4]:
data.duplicated().sum()
data.drop_duplicates(inplace=True)

In [5]:
data.dropna(how='any',inplace=True)

In [6]:
data.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type','listed_in(city)':'city'},inplace=True)

In [7]:
data['cost']=data['cost'].astype(str)
data['cost']=data['cost'].apply(lambda x: x.replace(',',''))
data['cost']=data['cost'].astype(float)

In [8]:
data['cost']=data['cost'].astype(str)
data['cost']=data['cost'].apply(lambda x: x.replace(',',''))
data['cost']=data['cost'].astype(float)

In [9]:
data = data.loc[data.rate !='NEW']
data= data.loc[data.rate !='-'].reset_index(drop=True)
data['rate']=data['rate'].apply(lambda x: x.replace('/5',''))
data['rate']=data['rate'].astype(float)

In [10]:
data.name = data.name.apply(lambda x:x.title())
data.online_order.replace(('Yes','No'),(True, False),inplace=True)
data.book_table.replace(('Yes','No'),(True, False),inplace=True)

In [11]:
restaurants = list(data['name'].unique())
data['Mean Rating'] = 0
for i in range(len(restaurants)):
    data['Mean Rating'][data['name'] == restaurants[i]] = data['rate'][data['name'] == restaurants[i]].mean()

<ipython-input-11-a0ff40cf8eaf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Mean Rating'][data['name'] == restaurants[i]] = data['rate'][data['name'] == restaurants[i]].mean()


In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (1,5))
data[['Mean Rating']] = scaler.fit_transform(data[['Mean Rating']]).round(2)

In [13]:
data['reviews_list']=data['reviews_list'].str.lower()

In [14]:
 data[['reviews_list','cuisines']]

,reviews_list,cuisines
0,"[('rated 4.0', 'rated\n a beautiful place to ...","North Indian, Mughlai, Chinese"
1,"[('rated 4.0', 'rated\n had been here for din...","Chinese, North Indian, Thai"
2,"[('rated 3.0', ""rated\n ambience is not that ...","Cafe, Mexican, Italian"
3,"[('rated 4.0', ""rated\n great food and proper...","South Indian, North Indian"
4,"[('rated 4.0', 'rated\n very good restaurant ...","North Indian, Rajasthani"
...,...,...
41232,"[('rated 4.0', 'rated\n ambience- big and spa...","North Indian, Continental"
41233,"[('rated 4.0', 'rated\n a fine place to chill...","Andhra, South Indian, Chinese, North Indian"
41234,"[('rated 5.0', ""rated\n food and service are ...",Continental
41235,"[('rated 4.0', 'rated\n nice and friendly pla...",Finger Food


In [15]:
import string
remove = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', remove))

data["reviews_list"] = data["reviews_list"].apply(lambda text: remove_punctuation(text))

In [16]:
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

data["reviews_list"] = data["reviews_list"].apply(lambda text: remove_stopwords(text))

In [18]:
import re
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

data["reviews_list"] = data["reviews_list"].apply(lambda text: remove_urls(text))

In [42]:
data.drop('menu_item',axis=1,inplace=True)

data.to_csv('data_cleaned.csv')

In [19]:
df_percent=data.sample(frac=0.1)

In [20]:
df_percent.set_index('name', inplace=True)

In [21]:
indices = pd.Series(df_percent.index)

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])

In [24]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [30]:
def recommend(name, cosine_similarities = cosine_similarities):

    recommend_restaurant = []

    idx = indices[indices == name].index[0]

    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)

    top30_indexes = list(score_series.iloc[0:31].index)

    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])

    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost']][df_percent.index == each].sample()))
    
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    
    return df_new


In [31]:
recommend('Bangalore Masala House')

TOP 10 RESTAURANTS LIKE Bangalore Masala House WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
The Pizza Bakery,"Italian, Pizza, Beverages",4.87,1200.0
House Of Commons,"Continental, Asian, North Indian",4.77,1000.0
Hammered,"North Indian, South Indian, Continental, Asian",4.65,1200.0
The Reservoire,"Continental, North Indian, Chinese, American",4.56,1300.0
Simon Says Brew Works,"Continental, American, Italian, Modern Indian,...",4.48,1400.0
Dyu Art Cafe,"Cafe, Italian, Fast Food",4.48,800.0
Float,"North Indian, Japanese",4.35,1500.0
Glen'S Bakehouse,"Bakery, Cafe, Italian, Desserts",4.27,600.0
Bangalore Masala House,"Cafe, Continental, Fast Food, Pizza, Italian, ...",4.04,500.0
Attic Lounge,Cafe,3.84,700.0


In [34]:
recommend('Burrito Boys')

TOP 10 RESTAURANTS LIKE Burrito Boys WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Mexican Burrito,"Mexican, Fast Food",3.84,450.0
Eggzotic,"North Indian, Chinese, Biryani, Fast Food",3.77,500.0
Sri Banashankari Donne Biriyani,Biryani,3.55,150.0
Agarwal Food Service,"North Indian, Chinese, Biryani",3.39,400.0
Sri Krishna Sagar,"South Indian, North Indian, Chinese, Street Fo...",3.26,400.0
Red Chilliez,"North Indian, Chinese, Seafood, Mangalorean",3.26,650.0
Nammura Donne Biriyani,"Mughlai, Biryani",3.20,400.0
Pot Biryani,"North Indian, Chinese, Biryani, Seafood",3.19,600.0
Sri Gurudev Darshini,South Indian,3.17,150.0
Just One More,"South Indian, Street Food",3.06,200.0


In [36]:
recommend('Kullukki')

TOP 10 RESTAURANTS LIKE Kullukki WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Belgian Waffle Factory,Desserts,4.93,400.0
Eggzotic,"North Indian, Chinese, Biryani, Fast Food",3.77,500.0
Madeena Hotel,"North Indian, Mughlai, Biryani",3.75,400.0
The Hidden Cafe,"North Indian, Fast Food",3.71,350.0
Mug N Bean,"Cafe, Continental, Italian, Burger",3.71,400.0
Cinnamon,"North Indian, Chinese, Biryani",3.62,550.0
Sri Banashankari Donne Biriyani,Biryani,3.55,150.0
Four Seasons Fast Food Center,"North Indian, Chinese, Biryani",3.49,400.0
Donne Biriyani Angadi Mane,"South Indian, Biryani",3.47,250.0
Nightfork,"Chinese, Biryani, Pizza, Italian, North Indian",3.45,500.0


NameError: name 'data' is not defined

from numpy import asarray
from numpy import save
cosine_similarities=asarray(cosine_similarities)
save('cos_sim.npy',cosine_similarities)
df_percent.to_csv('df_per.csv')
indices.to_csv('ind.csv')